In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 64
p1 = .5

class ResBlock(nn.Module):
    def __init__(self, in_size, out_size, kernel_size, stride, padding, padding_mode, encoder):
        super().__init__()
        if (encoder):
            self.conv1 = nn.Conv2d(in_size, in_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)
            self.conv2 = nn.Conv2d(in_size, out_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)
        else:
            self.conv1 = nn.ConvTranspose2d(in_size, in_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)
            self.conv2 = nn.ConvTranspose2d(in_size, out_size, kernel_size=kernel_size, stride=stride, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(in_size)
        self.batchnorm2 = nn.BatchNorm2d(out_size)
        
        self.dropout = nn.Dropout(p=p1)        

    def convblock(self, x):
        x = self.dropout(F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1))
        x = self.dropout(F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1))
        x = self.dropout(F.leaky_relu(self.batchnorm1(self.conv1(x)), negative_slope=0.1))
        x = self.dropout(F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1))
        return x
   
    def forward(self, x): 
        y = self.dropout(F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1))
        return y + self.convblock(x)
    
    
    
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.resblock1 = ResBlock(in_size=1, out_size=channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode, encoder=True)
        self.resblock2 = ResBlock(in_size=channels_base, out_size=channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode, encoder=True)
        
        self.resblock3 = ResBlock(in_size=channels_base * 2, out_size=channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode, encoder=False) 
        self.resblock4 = ResBlock(in_size=channels_base, out_size=channels_base //2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode, encoder=False) 
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
      
        
    def forward(self, input):
        # encoder
        x = F.avg_pool2d(self.resblock1(input), kernel_size=3, stride=2, padding=1)
        x = F.avg_pool2d(self.resblock2(x), kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = F.interpolate(self.resblock3(x), scale_factor=scale_factor)
        x = F.interpolate(self.resblock4(x), scale_factor=scale_factor)
        x = self.convtrans3(x)


        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]             640
       BatchNorm2d-2           [-1, 64, 32, 32]             128
           Dropout-3           [-1, 64, 32, 32]               0
            Conv2d-4            [-1, 1, 32, 32]              10
       BatchNorm2d-5            [-1, 1, 32, 32]               2
           Dropout-6            [-1, 1, 32, 32]               0
            Conv2d-7            [-1, 1, 32, 32]              10
       BatchNorm2d-8            [-1, 1, 32, 32]               2
           Dropout-9            [-1, 1, 32, 32]               0
           Conv2d-10            [-1, 1, 32, 32]              10
      BatchNorm2d-11            [-1, 1, 32, 32]               2
          Dropout-12            [-1, 1, 32, 32]               0
           Conv2d-13           [-1, 64, 32, 32]             640
      BatchNorm2d-14           [-1, 64,

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.02298395 (0.20458790), PSNR 16.38575363 (12.50010256), SSIM 0.29666597 (0.16441867)
Finished training epoch 0


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 13 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 23 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 67 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 37 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 47 pixels
  return func(*args, **kwargs)

Validate: MSE 0.11077912 (0.16378820), PSNR 9.55542088 (8.09556947), SSIM 0.16447586 (0.14967496)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.01164067 (0.01584853), PSNR 19.34021950 (18.11935042), SSIM 0.60446495 (0.51036698)
Finished training epoch 1


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 3 pixels
  return func(*args, **kwargs)


Validate: MSE 0.01480264 (0.01917540), PSNR 18.29660606 (17.33544176), SSIM 0.41312543 (0.43036471)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00652966 (0.00846969), PSNR 21.85109520 (20.76914165), SSIM 0.65759146 (0.65406916)
Finished training epoch 2
Validate: MSE 0.00871071 (0.00942068), PSNR 20.59946060 (20.36184202), SSIM 0.53105396 (0.58119476)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00473071 (0.00580127), PSNR 23.25073242 (22.39227225), SSIM 0.71855307 (0.69805947)
Finished training epoch 3
Validate: MSE 0.00521380 (0.00490080), PSNR 22.82845497 (23.16746544), SSIM 0.61524069 (0.67770632)
Finished validation.
Starting training epoch 4
Epoch: 4, MSE 0.00438999 (0.00447827), PSNR 23.57536697 (23.50783464), SSIM 0.71843350 (0.72088116)
Finished training epoch 4
Validate: MSE 0.00446837 (0.00406752), PSNR 23.49850464 (23.96795947), SSIM 0.65517616 (0.72331189)
Finished validation.
Starting training epoch 5
Epoch: 5, MSE 0.00343010 (0.003802

Validate: MSE 0.00365860 (0.00298581), PSNR 24.36684799 (25.33201301), SSIM 0.70585680 (0.77469403)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00260145 (0.00242445), PSNR 25.84784317 (26.17912525), SSIM 0.76781887 (0.77121540)
Finished training epoch 32
Validate: MSE 0.00354940 (0.00287913), PSNR 24.49845123 (25.48503251), SSIM 0.70637715 (0.77532254)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00265947 (0.00242841), PSNR 25.75204086 (26.17054623), SSIM 0.77297723 (0.77091516)
Finished training epoch 33
Validate: MSE 0.00351838 (0.00284886), PSNR 24.53656960 (25.52923349), SSIM 0.70689815 (0.77682267)
Finished validation.
Starting training epoch 34
Epoch: 34, MSE 0.00338086 (0.00243125), PSNR 24.70972061 (26.16646742), SSIM 0.74665892 (0.77053570)
Finished training epoch 34
Validate: MSE 0.00362521 (0.00301046), PSNR 24.40667152 (25.28974704), SSIM 0.70365179 (0.77227972)
Finished validation.
Starting training epoch 35
Epoch: 35, MSE 0.0020805

Validate: MSE 0.00360118 (0.00295778), PSNR 24.43554878 (25.37264480), SSIM 0.70379961 (0.77426385)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00236495 (0.00234235), PSNR 26.26177597 (26.32501927), SSIM 0.77038807 (0.77220074)
Finished training epoch 62
Validate: MSE 0.00355652 (0.00295101), PSNR 24.48974800 (25.37943025), SSIM 0.70438004 (0.77152437)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00190790 (0.00233790), PSNR 27.19444656 (26.33620090), SSIM 0.76982844 (0.77283367)
Finished training epoch 63
Validate: MSE 0.00335068 (0.00285914), PSNR 24.74867058 (25.51171765), SSIM 0.70415473 (0.77405747)
Finished validation.
Starting training epoch 64
Epoch: 64, MSE 0.00238103 (0.00233907), PSNR 26.23235893 (26.32989771), SSIM 0.74867421 (0.77307764)
Finished training epoch 64
Validate: MSE 0.00356802 (0.00310104), PSNR 24.47572899 (25.15987335), SSIM 0.70111650 (0.77219770)
Finished validation.
Starting training epoch 65
Epoch: 65, MSE 0.0021857

Validate: MSE 0.00392166 (0.00314493), PSNR 24.06529808 (25.10954688), SSIM 0.70262647 (0.77354343)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00253168 (0.00231861), PSNR 25.96590233 (26.37119338), SSIM 0.74380672 (0.77220820)
Finished training epoch 92
Validate: MSE 0.00400061 (0.00327264), PSNR 23.97873306 (24.95127283), SSIM 0.70110154 (0.77287374)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00290818 (0.00230645), PSNR 25.36378670 (26.39566087), SSIM 0.75511944 (0.77282271)
Finished training epoch 93
Validate: MSE 0.00499918 (0.00395332), PSNR 23.01101112 (24.16656037), SSIM 0.69480711 (0.76918093)
Finished validation.
Starting training epoch 94
Epoch: 94, MSE 0.00278783 (0.00230243), PSNR 25.54733467 (26.40116833), SSIM 0.75569582 (0.77348436)
Finished training epoch 94
Validate: MSE 0.00355984 (0.00290379), PSNR 24.48569489 (25.44813590), SSIM 0.70368707 (0.77443878)
Finished validation.
Starting training epoch 95
Epoch: 95, MSE 0.0025094

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00508175 (0.00409459), PSNR 22.93986320 (24.01282807), SSIM 0.69737864 (0.76919520)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()